# Stock Portfolio Optimization Tool Using AI

In [7]:
import pandas as pd

dataFileName = input("Please enter the name of the data file.")
print(dataFileName)
stock_data = pd.read_csv("Stocks/"+dataFileName+".csv")

stock_data.head()

Please enter the name of the data file.prices
prices


,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [8]:
stock_data.isnull()

,date,symbol,open,close,low,high,volume
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
851259,False,False,False,False,False,False,False
851260,False,False,False,False,False,False,False
851261,False,False,False,False,False,False,False
851262,False,False,False,False,False,False,False


In [9]:
stock_data.tail(n=2)

,date,symbol,open,close,low,high,volume
851262,2016-12-30 00:00:00,AIV,44.730000,45.450001,44.410000,45.59,1380900.0
851263,2016-12-30 00:00:00,FTV,54.200001,53.630001,53.389999,54.48,705100.0


In [10]:
stock_data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid", color_codes=True)

sns.set(rc={'figure.figsize':(11.6,8.27)})

sns.countplot('open',data=stock_data,hue='close')

sns.despine(offset=10, trim=True)